<a href="https://colab.research.google.com/github/anushka-220/Undergraduate-Project/blob/main/Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q condacolab

In [1]:
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [3]:
!conda create -n py3918 python=3.9.18 -y


Channels:
 - conda-forge
Platform: linux-64
Solving environment: - \ done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.2
    latest version: 25.1.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local/envs/py3918

  added / updated specs:
    - python=3.9.18


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2025.1.31  |       hbcca054_0         154 KB  conda-forge
    liblzma-5.6.4              |       hb9d3cd8_0         109 KB  conda-forge
    liblzma-devel-5.6.4        |       hb9d3cd8_0         370 KB  conda-forge
    libsqlite-3.48.0           |       hee588c1_1         858 KB  conda-forge
    ncurses-6.5                |       h2d0b736_3         871 KB  conda-forge
    pip-25.0                   |     pyh8b19718_0         1.2 MB  conda-forge
   

In [5]:
!source activate py3918

In [6]:
!conda run -n py3918 python --version

Python 3.9.18

